# **DATA PREPROCESSING**

---

## **1. Import libraries**

In [1]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from data_processing import (
    load_data,
    handle_missing,
    feature_engineering,
    normalize_features,
    parse_date_column,
    target_encoding,
    label_encoding,
    select_features,
    pca_reduction,
    train_test_split
)

## **2. Load Raw Data**

In [2]:
df = load_data("../data/raw/weatherAUS.csv")

Đã load dữ liệu: 145460 hàng, 23 cột


## **3. Handle missing value**

Xử lý được thiết kế dựa trên đặc tính lý hóa của dữ liệu khí tượng và mục tiêu bảo toàn thông tin vật lý tối đa cho mô hình dự báo.

#### **3.1. Biến mục tiêu (RainTomorrow)**
- **Chiến lược:** Loại bỏ các dòng giá trị thiếu (Drop NA).
- **Lý do:** Đây là nhãn (label) cho bài toán học máy giám sát. Việc điền giá trị giả định cho biến mục tiêu sẽ làm sai lệch hoàn toàn bản chất của dữ liệu và giảm độ tin cậy của mô hình khi đánh giá.

#### **3.2. Biến phân loại nhị phân (RainToday)**
- **Chiến lược:** Mã hóa số (0/1) và điền NA bằng giá trị đại diện (giá trị 2).
- **Lý do:** Thay vì xóa bỏ các dòng thiếu RainToday, việc dùng giá trị '2' giúp mô hình học được mối liên hệ giữa trạng thái "không quan trắc được" (missing data) và khả năng mưa ngày hôm sau.

#### **3.3. Nhóm biến có tỷ lệ thiếu cao (Evaporation, Sunshine, Cloud)**
- **Chiến lược:** Tạo cột Missing Indicator Flags (`_Missing`).
- **Lý do:** Sự thiếu hụt dữ liệu ở các cột này (như nắng, mây) thường không ngẫu nhiên mà do điều kiện thời tiết khắc nghiệt. Việc tạo Flag giúp mô hình phân biệt được đâu là giá trị dự đoán từ máy tính và đâu là dữ liệu thực tế từng bị trống, giữ lại thông tin về trạng thái khí quyển ban đầu.

#### **3.4. Nhóm biến số (Numeric Variables)**
- **Chiến lược:** Áp dụng **Iterative Imputer (MICE)** với `max_iter=20`.
- **Lý do:** Thay vì dùng trung bình (Mean) khiến dữ liệu bị "phẳng", MICE dự đoán giá trị thiếu dựa trên mối tương quan đa biến (ví dụ: dự đoán độ ẩm dựa trên nhiệt độ và áp suất). Điều này đảm bảo tính logic và nhất quán giữa các chỉ số khí tượng trong cùng một thời điểm.


#### **3.5. Nhóm biến phân loại (Categorical Variables)**
- **Chiến lược:** Thay thế bằng giá trị xuất hiện nhiều nhất (Mode Imputation).

In [3]:
df = handle_missing(df)

Xử lý missing hoàn tất


## **4. Feature engineering**

Quá trình xử lý được xây dựng dựa trên nguyên lý: "Trong khí tượng, sự biến động quan trọng hơn trạng thái tĩnh". Thay vì chỉ cung cấp cho mô hình các con số thô tại một thời điểm, nhóm tiến hành tạo ra các đặc trưng phản ánh động lực học của bầu khí quyển bằng cách xây dựng các biến Delta như sau: 

#### **4.1. Sự thay đổi áp suất (PressureChange)**:
* **Chiến lược:** Tạo cột dữ liệu mới `PressureChange` bằng cách lấy hiệu của 2 biến `Pressure3pm` và `Pressure9am`.
* **Lý do:** Áp suất giảm thường báo hiệu một hệ thống áp thấp đang tới, là dấu hiệu cốt lõi của mưa/bão.

#### **4.2. Biên độ nhiệt (TempRange)**: 
* **Chiến lược:** Tạo cột dữ liệu mới `TempRange` bằng cách lấy hiệu của 2 biến `MaxTemp` và `MinTemp`.
* **Lý do:** Chênh lệch nhiệt độ giữa ngày và đêm ảnh hưởng đến quá trình ngưng tụ hơi nước.

#### **4.3. Sự thay đổi độ ẩm (HumidityChange)**: 
* **Chiến lược:** Tạo cột dữ liệu mới `HumidityChange` bằng cách lấy hiệu của 2 biến `Humidity3pm` và `Humidity9am`.
* **Lý do:** Phản ánh tốc độ tích tụ hơi nước trong không khí từ sáng đến chiều.

In [4]:
df = feature_engineering(df)

## **5. Normalization**

Áp dụng các kỹ thuật chuyển đổi khác nhau cho từng nhóm biến dựa trên phân bố xác suất và độ nhạy cảm của chúng đối với các giá trị cực đoan.

#### **5.1. Biến lượng mưa (Rainfall)**
- **Chiến lược:** `clip(lower=0)` -> `log1p` -> `RobustScaler`.
- **Lý do:** 
    - **Clip:** Loại bỏ các giá trị âm cực nhỏ phát sinh ngoài ý muốn từ quá trình dự đoán của MICE, đảm bảo tính hợp lệ cho hàm Log.
    - **Log1p:** Lượng mưa thường có phân bố lệch phải cực hạn (nhiều giá trị 0 và một vài giá trị rất lớn). Phép biến đổi Log giúp kéo giãn các giá trị nhỏ và nén các giá trị lớn, đưa phân bố về gần dạng chuẩn hơn.
    - **RobustScaler:** Vì chúng ta quyết định giữ lại các giá trị ngoại lai (outliers) đại diện cho mưa cực đoan, RobustScaler sử dụng Trung vị (Median) và Khoảng biến thiên nội phần tư (IQR) để chuẩn hóa thay vì Trung bình, giúp thang đo không bị kéo lệch bởi các cơn mưa lớn.


#### **5.2. Nhóm nhiệt độ và Độ ẩm (Temperature & Humidity)**
- **Chiến lược:** `StandardScaler` (Z-score normalization).
- **Lý do:** Các biến này thường có phân bố tự nhiên gần với phân bố hình chuông (Normal Distribution). StandardScaler đưa dữ liệu về trạng thái có trung bình bằng 0 và độ lệch chuẩn bằng 1, giúp các mô hình học máy (như Logistic Regression hay SVM) hội tụ nhanh hơn và hoạt động ổn định hơn.


#### **5.3. Nhóm biến khí động học và bức xạ (Wind, Pressure, Sunshine, Cloud...)**
- **Chiến lược:** `MinMaxScaler`.
- **Lý do:** 
    - Đưa tất cả các biến về cùng một thang đo cố định từ **0 đến 1**. 
    - Chiến lược này bảo toàn chính xác hình dạng phân bố gốc của dữ liệu (như các biến Cloud thường có giá trị rời rạc hoặc Sunshine có giới hạn vật lý rõ rệt). Việc đưa về cùng biên độ [0, 1] giúp mô hình không bị thiên kiến (bias) vào các biến có đơn vị đo lớn hơn (như áp suất ~1000 hPa so với độ che phủ mây ~8 oktas).

In [5]:
df = normalize_features(df)

Hoàn thành chuẩn hóa dữ liệu


## **6. Convert type of column `Date`**

Chuyển đổi dữ liệu thô dạng chuỗi (String) thành các thuộc tính có ý nghĩa về mặt chu kỳ và thời gian, giúp mô hình hiểu được yếu tố mùa vụ của khí tượng.

#### **6.1. Chuyển đổi định dạng Datetime**
- **Chiến lược:** Sử dụng `pd.to_datetime` với `errors='coerce'`.
- **Lý do:** Chuyển đổi cột ngày tháng từ dạng văn bản sang đối tượng thời gian chuẩn để máy tính có thể thực hiện các phép toán trích xuất. Tham số `errors='coerce'` đảm bảo nếu có dữ liệu ngày tháng bị sai định dạng, chúng sẽ được chuyển thành `NaT` (Not a Time) thay vì làm dừng chương trình.

#### **6.2. Trích xuất thuộc tính Tháng (Month) và Năm (Year)**
- **Chiến lược:** Tách cột `Date` thành hai cột số riêng biệt là `Month` và `Year`.
- **Lý do:** 
    - **Year:** Giúp mô hình nhận diện xu hướng biến đổi khí hậu dài hạn qua các năm.
    - **Month:** Đây là biến số cực kỳ quan trọng trong dự báo thời tiết vì các hiện tượng như mưa, áp suất thường lặp lại theo chu kỳ hàng năm. Việc chuyển về dạng số (1-12) giúp mô hình dễ dàng tính toán mối tương quan này.

#### **6.3. Phân loại Mùa (Season)**
- **Chiến lược:** Áp dụng hàm `get_season` dựa trên lịch khí tượng của Úc.
- **Lý do:** 
    - **Đặc thù địa lý:** Úc nằm ở Nam bán cầu, nên các mùa sẽ ngược lại với Bắc bán cầu (ví dụ: tháng 12, 1, 2 là mùa Hè). 
    - **Ý nghĩa mô hình:** Việc gom nhóm các tháng thành mùa (`Summer`, `Autumn`, `Winter`, `Spring`) tạo ra một biến phân loại cấp cao hơn. Mùa là yếu tố chủ đạo quyết định kiểu thời tiết (mùa Đông thường có front lạnh, mùa Hè thường có dông bão), giúp mô hình giảm bớt sự phức tạp so với việc chỉ nhìn vào từng tháng riêng lẻ.

In [6]:
df = parse_date_column(df)

Đã xử lý cột Date


## **7. Target encoding**

Kỹ thuật Target Encoding được áp dụng cho biến `Location` để mã hoá các khu vực địa lí.
#### **7.1. Chiến lược:**
Quy trình thực hiện Target Encoding diễn ra theo 3 bước logic:
* **Tính toán giá trị đại diện**: Với mỗi địa điểm cụ thể, chúng ta tính giá trị trung bình (Mean) của biến mục tiêu `RainTomorrow`. Con số này chính là tỷ lệ ngày mưa lịch sử tại trạm đó.
* **Ánh xạ (Mapping)**: Thay thế tên địa điểm (ví dụ như "Sydney") bằng con số xác suất vừa tính được (ví dụ: 0.26). Biến mới này được đặt tên là `Location_Risk`.
* **Loại bỏ biến gốc**: Xóa cột `Location` để đảm bảo mọi đầu vào cho mô hình đều là dạng số.
#### **7.2. Lý do:**
Biến `Location` là một biến phân loại có độ băm cao với khoảng 49 địa điểm khác nhau, vì vậy mà áp dụng kỹ thuật này giúp nén toàn bộ thông tin địa lý vào duy nhất 1 cột số, đại diện cho "Mức độ rủi ro" hoặc "Xác suất mưa trung bình" của từng trạm đo. Điều này giúp mô hình nắm bắt được trực tiếp mối quan hệ giữa địa điểm và mục tiêu dự báo.

In [7]:
df = target_encoding(df)

## **8. Label encoding**

Chuyển đổi các biến phân loại dạng văn bản (Categorical) sang dạng số nguyên để các thuật toán học máy có thể thực hiện các phép toán toán học.

#### **8.1. Lựa chọn cột mã hóa (Wind Directions)**
- **Chiến lược:** Áp dụng cho các cột hướng gió (`WindGustDir`, `WindDir9am`, `WindDir3pm`) và `Season`.
- **Lý do:** Đây là các biến định danh đại diện cho 16 hướng gió (N, S, E, W, NE, v.v.) và 4 mùa trong năm. Máy tính không thể xử lý trực tiếp các ký tự chữ cái, do đó việc chuyển đổi sang số là bước bắt buộc để đưa vào các mô hình học máy.

#### **8.2. Sử dụng LabelEncoder**
- **Chiến lược:** Chuyển đổi mỗi giá trị chữ thành một số nguyên duy nhất (ví dụ: 'E' -> 0, 'ENE' -> 1, 'ESE' -> 2...).
- **Lý do:** Đối với các biến hướng gió có nhiều nhóm (16 hướng), Label Encoding là giải pháp gọn nhẹ hơn so với One-Hot Encoding (vốn sẽ tạo ra thêm quá nhiều cột mới gây loãng dữ liệu - "Curse of Dimensionality"). Cách tiếp cận này đặc biệt hiệu quả cho các mô hình dạng cây (như Decision Tree, Random Forest) vì chúng có khả năng xử lý tốt các biến số nguyên này.


#### **8.3. Ép kiểu dữ liệu (astype(str))**
- **Chiến lược:** Sử dụng `.astype(str)` trước khi fit dữ liệu.
- **Lý do:** Đảm bảo dữ liệu đầu vào cho bộ mã hóa luôn đồng nhất là kiểu chuỗi. Điều này giúp tránh lỗi chương trình trong trường hợp có dữ liệu nhiễu hoặc các giá trị không xác định sót lại, đảm bảo quá trình mã hóa diễn ra ổn định.

In [8]:
df = label_encoding(df)

Hoàn thành Label Encoding


## **9. Select Features**

#### **9.1. Nhóm Nhiệt độ (Temp9am, Temp3pm)**
* **Chiến lược:** Loại bỏ `Temp9am` và `Temp3pm`, giữ lại `MaxTemp` và `TempRange`.
* **Lý do:**
    * `Temp9am` và `Temp3pm` có tương quan cực cao với `MaxTemp`. Trong dự báo thời tiết, nhiệt độ tối đa (`MaxTemp`) đại diện cho mức năng lượng nhiệt cao nhất trong ngày, mang nhiều thông tin hơn các điểm nhiệt độ tức thời.
    * Vì vậy nên biến `TempRange` được tạo ra để bao hàm thông tin về sự biến thiên nhiệt độ. Nếu giữ cả `Temp9am` và `Temp3pm`, mô hình sẽ không ổn vì cùng một thông tin về nhiệt độ được lặp lại ở nhiều cột khác nhau.

#### **9.2. Nhóm Áp suất (Pressure9am)**
* **Chiến lược:** Loại bỏ `Pressure9am`, giữ lại `Pressure3pm` và `PressureChange`.
* **Lý do:**
    * Việc giữ cả 3 biến sẽ gây ra hiện tượng đa cộng tuyến hoàn hảo. Đối với mô hình Logistic Regression, điều này làm cho các hệ số hồi quy không ổn định và mất khả năng giải thích.
    * Ngoài ra, trong việc dự báo mưa cho ngày mai thì các chỉ số buổi chiều (3pm) thường có giá trị dự báo cao hơn các chỉ số buổi sáng (9am) vì chúng gần với thời điểm chuyển giao sang ngày hôm sau hơn.

In [9]:
df = select_features(df)

Đã loại bỏ các biến: ['Temp9am', 'Temp3pm', 'Pressure9am']


## **10. Split into train and test**

Đảm bảo tính khách quan trong việc đánh giá hiệu suất của mô hình, tránh hiện tượng học vẹt (Memorization) và đảm bảo khả năng tổng quát hóa (Generalization).

#### **10.1. Phân chia tập huấn luyện và tập kiểm tra (Ratio 80/20)**
- **Chiến lược:** Sử dụng tỷ lệ `test_size=0.2`.
- **Lý do:** Tỷ lệ 80/20 là một tiêu chuẩn vàng trong học máy. Tập huấn luyện (80%) đủ lớn để mô hình học được các quy luật thời tiết phức tạp, trong khi tập kiểm tra (20%) đủ đại diện để đánh giá khả năng dự báo của mô hình trên dữ liệu thực tế hoàn toàn mới.


#### **10.2. Xáo trộn dữ liệu ngẫu nhiên (Random Permutation)**
- **Chiến lược:** Sử dụng `np.random.permutation(n)`.
- **Lý do:** Dữ liệu thời tiết gốc thường được sắp xếp theo thời gian hoặc địa điểm (ví dụ: hết ngày này sang ngày khác, hết trạm này sang trạm khác). Nếu không xáo trộn, tập Test có thể chỉ toàn dữ liệu của những năm cuối hoặc của một vài thành phố nhất định. Việc xáo trộn giúp phân phối đều các đặc trưng mùa vụ và địa lý vào cả hai tập dữ liệu.

#### **10.3. Tính nhất quán trong thực nghiệm (Random State)**
- **Chiến lược:** Thiết lập `np.random.seed(random_state)`.
- **Lý do:** Đảm bảo tính tái lập (Reproducibility). Trong quá trình thử nghiệm nhiều mô hình khác nhau, việc giữ cố định tập Train và tập Test giúp chúng ta so sánh công bằng hiệu suất của các thuật toán (ví dụ: so sánh Random Forest với SVM trên cùng một bộ dữ liệu đầu vào).

#### **10.4. Bảo toàn tính nguyên bản (Deep Copy)**
- **Chiến lược:** Sử dụng phương thức `.copy()`.
- **Lý do:** Tránh lỗi `SettingWithCopyWarning` trong Pandas. Việc tạo bản sao độc lập (`train_df` và `test_df`) đảm bảo rằng mọi thay đổi hoặc tính toán sau này trên tập huấn luyện sẽ không vô tình làm thay đổi dữ liệu gốc hoặc ảnh hưởng đến tập kiểm tra, giữ cho quá trình thực nghiệm luôn sạch và minh bạch.

In [11]:
train_df, test_df = train_test_split(df)

Chia dữ liệu hoàn tất


## **11. Save processed data**

In [12]:
train_df.to_csv("../data/processed/train.csv", index=False)
test_df.to_csv("../data/processed/test.csv", index=False)
df.to_csv("../data/processed/data.csv", index=False )
print("Lưu thành công")

Lưu thành công
